In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd 

class OPTIMISER:
    
    def __init__(self):
        pass
    
    def example(self):
        try:
            m = gp.Model("mip1")

            # Import data 
            path = f"C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\Railway services-2024.xlsx"
            df = pd.read_excel(path)
            df = df.dropna(subset=['Trip'])
            #print(df.head())
            demand = df.set_index('Trip')['Demand(μ)'].to_dict()  # Extract demand
            demand = {float(key): value for key, value in demand.items()} # Convert keys in the demand dictionary to float

            #print(demand.keys())
            #print(df['Trip'].unique())
            platform_length = df.set_index('Trip')['Line'].apply(lambda x: 300 if x != 400 else 200).to_dict()  # Set platform length based on line
            s = {'OC': 620, 'OH': 420}
            l = {'OC': 100, 'OH': 70}
            f = {'OC': 260000, 'OH': 210000}

            keys = s.keys()
        
            n = {}
            for u in keys:  # Define the first index range
                for c in range(200):  # Define the second index range
                    n[u, c] = m.addVar(vtype=GRB.INTEGER, lb=0, name=f"n[{u},{c}]") #nonnegativity included here 
            
            m.setObjective(gp.quicksum(n[u, c] * f[u] for u in keys for c in range(200)), GRB.MINIMIZE)
            
            for c in range(200):
                trip = c + 1
                #print("Demand dictionary keys:", demand.keys())
                #print("Current trip value:", trip)
                m.addConstr(gp.quicksum(n[u, c] * s[u] for u in keys) >= demand[float(trip)])
                m.addConstr(gp.quicksum(n[u, c] * l[u] for u in keys) <= platform_length[trip])

            
            for u in keys:
                for k in keys:
                    if u != k:
                        m.addConstr(gp.quicksum(n[u,c] - n[k,c] for c in range(200)) <= gp.quicksum(n[k, c] for c in range(200)) * 0.25)
                    


            
            m.optimize()
            
            for v in m.getVars():
                print("%s %g" % (v.varName, v.x))
                
            print("Obj: %g" % m.objVal)
            
        except gp.GurobiError as e:
            print("Error code " + str(e.errno) + ": " + str(e))
        except AttributeError:
            print("Encountered an attribute error")


if __name__ == "__main__":
    optimizer = OPTIMISER()
    optimizer.example()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 402 rows, 400 columns and 1600 nonzeros
Model fingerprint: 0xbfb5a7c0
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve time: 0.01s
Presolved: 402 rows, 400 columns, 1600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.268188e+03   0.000000e+00      0s
     273    6.1147590e+07   0.000000e+00   0.000000e+00      0s

Solved in 273 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.114758996e+07
n[OC,0] 0
n[OC,1] 0
n[OC,2] 0
n[OC,3] 0
n[OC,4] 0
n[OC,5] 0.89562
n[OC,6] 0
n[OC,7] 0
n[OC,8] 1.00484
n[OC,9] 1.75806
n[OC,10] 0.248387
n[OC,11] 